In [1]:
import numpy as np
import pandas as pd

Ввод данных

In [2]:
n = 10
Af = np.matrix([[1 if i == j else 1.0/(i+j+2) for j in range(n)] + [1/(i+1)] for i in range(n)])
A = np.matrix([[1 if i == j else 1.0/(i+j+2) for j in range(n)] for i in range(n)])
f = np.matrix([[1.0/(i+1)] for i in range(n)])

In [3]:
D = np.diag(np.diag(A))
L = np.tril(A, k = -1)
U = np.triu(A, k = 1)

# Метод Зейделя

In [4]:
def seidel(D, L, U, iteration, eps=1e-10):
    n = len(D)
    x_0 = np.matrix([[0]]*n)
    x_k = x_0[:]
    for k in range(iteration):
        LD_inv = np.matrix(np.linalg.inv(L + D))
        x_k1 = LD_inv*f - (LD_inv*U)*x_k 
        if all(abs(x_k1.tolist()[i][0] - x_k[i]) < eps for i in range(n)):
            return [x[0] for x in x_k1.tolist()]
        x_k = x_k1[:]
    raise ValueError('Метод не сходится')

Здесь критерием останова служит условие достижения разности решений текущей и при

# Метод Гаусса

In [5]:
def gauss(A):
    A = A.tolist()
    n = len(A)

    for i in range(0, n):
        # Ищем максимум в каждом столбце
        maxEl = abs(A[i][i])
        maxRow = i
        for k in range(i+1, n):
            if abs(A[k][i]) > maxEl:
                maxEl = abs(A[k][i])
                maxRow = k

        # Меняем строку с максимумом с текущей строкой
        for k in range(i, n+1):
            tmp = A[maxRow][k]
            A[maxRow][k] = A[i][k]
            A[i][k] = tmp

        # Обнуляем все эдементы текущего столбца ниже этой строки
        for k in range(i+1, n):
            c = -A[k][i]/A[i][i]
            for j in range(i, n+1):
                if i == j:
                    A[k][j] = 0
                else:
                    A[k][j] += c * A[i][j]
    
    return solve_equat(A, n)

#Поиск решения 
def solve_equat(A,n):
    x = [0 for i in range(n)]
    for i in range(n-1, -1, -1):
        x[i] = A[i][n]/A[i][i]
        for k in range(i-1, -1, -1):
            A[k][n] -= A[k][i] * x[i]
    return x

Обработка результатов

In [6]:
seid_x = seidel(D, L, U, 50)
seid_xm = np.matrix([[seid_x[i]] for i in range(len(seid_x))])

In [7]:
gaus_x = gauss(Af)
gaus_xm = np.matrix([[gaus_x[i]] for i in range(len(gaus_x))])

# Печать невязок

In [8]:
def f_error(A, gaus_xm, seid_xm, f):
    fseid = A * seid_xm
    fgaus = A * gaus_xm
    df_seid = [abs(x[0]) for x in (f - fseid).tolist()]
    df_gaus = [abs(x[0]) for x in (f - fgaus).tolist()]
    data = [list(x) for x in list(zip(df_gaus, df_seid))]
    return data

In [9]:
output = f_error(A, gaus_xm, seid_xm, f)
table = pd.DataFrame(columns=['Gauss method', 'Seiden method'], data=output)

In [10]:
table

,Gauss method,Seiden method
0,2.220446e-16,3.518963e-12
1,0.000000e+00,1.242395e-12
2,0.000000e+00,2.040590e-12
3,0.000000e+00,1.878220e-12
4,2.775558e-17,1.483230e-12
5,2.775558e-17,1.072170e-12
6,0.000000e+00,7.107370e-13
7,2.775558e-17,4.140299e-13
8,0.000000e+00,1.797590e-13
9,2.775558e-17,1.387779e-17


# Поиск максимального и минимального собственного значения

In [11]:
lambdas = np.linalg.eig(A)[0]
lmin = min(lambdas)
lmax = max(lambdas)

In [12]:
lmin, lmax

(0.6579597538101791, 2.048359926977445)

# Определение числа обусловленности

In [13]:
cond = np.linalg.cond(A)

In [14]:
cond

3.113199424608568